In [ ]:
import keras
import keras.objectives
import numpy as np
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import util


%matplotlib inline

In [ ]:
dataset = np.load('./datasets/v0_eigens.npz')

train_data_size = dataset['train_eigens'].shape[0]
valid_data_size = train_data_size / 5
train_data_size = train_data_size - valid_data_size

indices = np.arange(train_data_size + valid_data_size)

# np.random.shuffle(indices)

train_data = dataset['train_eigens'][indices[:train_data_size]]
valid_data = dataset['train_eigens'][indices[train_data_size:]]

train_eigens = train_data[:, :-28]
train_labels = train_data[:, -28:]
valid_eigens = valid_data[:, :-28]
valid_labels = valid_data[:, -28:]
issue_eigens = dataset['issue_eigens'][:, :-28]

print 'train_eigens.shape = {}'.format(train_eigens.shape)
print 'train_labels.shape = {}'.format(train_labels.shape)
print 'valid_eigens.shape = {}'.format(valid_eigens.shape)
print 'valid_labels.shape = {}'.format(valid_labels.shape)

In [ ]:
def build_model():
    """
    """
    eigens = tf.placeholder(tf.float32, shape=(None, 896))
    labels = tf.placeholder(tf.float32, shape=(None, 28))

    tensors = keras.layers.Dense(128, activation='relu')(eigens)
    tensors = keras.layers.Dense(128, activation='relu')(tensors)
    predictions = keras.layers.Dense(28, activation='sigmoid')(tensors)

    loss = tf.reduce_mean(keras.objectives.binary_crossentropy(labels, predictions))
    
    trainer = tf.train \
        .AdamOptimizer(learning_rate=0.00005, beta1=0.5) \
        .minimize(loss)
        
    return {
        'eigens': eigens,
        'labels': labels,
        'predictions': predictions,
        'loss': loss,
        'trainer': trainer,
    }

In [ ]:
model = build_model()

batch_size = 256

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for step, eigens, labels in util.mini_batches(train_eigens, train_labels, batch_size, False):
        feed_dict = {
            model['eigens']: eigens,
            model['labels']: labels,
        }
        
        fetches = {
            'trainer': model['trainer'],
            'loss': model['loss'],
        }

        fetched = session.run(fetches, feed_dict=feed_dict)

        if step % 5000 == 0:
            print 'loss[{:>8}]: {}'.format(step, fetched['loss'])

        if step == 50000:
            break

    feed_dict = {model['eigens']: valid_eigens}
    
    valid_guesss = session.run(model['predictions'], feed_dict=feed_dict)

    auc = util.auc(valid_guesss.flatten(), valid_labels.flatten())

    print 'auc[after]: {}'.format(auc)
    
    
    feed_dict = {model['eigens']: issue_eigens}
    
    predictions = session.run(model['predictions'], feed_dict=feed_dict)
    
    util.write_result('dense.csv', predictions)